In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from peft import PeftModel
from transformers import BitsAndBytesConfig
import torch
import random
from diffusers import HunyuanVideoPipeline, HunyuanVideoTransformer3DModel
import datetime

# -------------------
# Hugging Face Login
# -------------------
login("<YOUR_HUGGINGFACE_TOKEN>")

# -------------------
# Model Setup
# -------------------
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quant_config, torch_dtype=torch.bfloat16, device_map="auto")

# -------------------
# Ask Mistral to Suggest Creative Elements
# -------------------
def ask_mistral(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**input_ids, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

categories = {
    "characters": "Give a list of 10 fun and creative cartoon-style characters for kids.",
    "actions": "List 10 imaginative actions a character can do in a kids animation.",
    "settings": "Suggest 10 colorful and playful settings for a kids animation.",
    "styles": "Provide 10 fun visual styles for kids animation (e.g., cartoon, 3D, watercolor)."
}

creative_elements = {}
for key, prompt in categories.items():
    response = ask_mistral(prompt)
    creative_elements[key] = [line.strip("- ") for line in response.split("\n") if line.strip() and not line.startswith("<")]

# -------------------
# Generate Series of Prompts with Same Character
# -------------------
def generate_prompt_with_character(character, actions, settings, styles):
    return f"{character} {random.choice(actions)} {random.choice(settings)} {random.choice(styles)}"


# -------------------
# Generate Metadata for YouTube Upload Using Mistral
# -------------------
metadata_prompt = f"""Generate YouTube metadata for a kids animation video.
Use the following prompts for context: {prompts_list}

Format your reply as:
Title: <title>
Description: <engaging description>
Tags: tag1, tag2, tag3, ...
"""
metadata_response = ask_mistral(metadata_prompt)

# Parse structured metadata
lines = metadata_response.split("\n")
title = ""
description = ""
tags = []

for line in lines:
    if line.lower().startswith("title:"):
        title = line.split(":", 1)[1].strip()
    elif line.lower().startswith("description:"):
        description = line.split(":", 1)[1].strip()
    elif line.lower().startswith("tags:"):
        tags = [tag.strip() for tag in line.split(":", 1)[1].split(",")]

# -------------------
# Final Video Metadata Assignment
# -------------------
date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
VIDEO_FILE = f"output_{date_str}.mp4"
VIDEO_TITLE = title
VIDEO_DESCRIPTION = description
VIDEO_CATEGORY_ID = "22"  # People & Blogs
VIDEO_TAGS = tags


# -------------------
# Generate Series of Prompts for < 60 sec Video
# -------------------
main_character = random.choice(creative_elements["characters"])
actions = creative_elements["actions"]
settings = creative_elements["settings"]
styles = creative_elements["styles"]

prompts_list = [generate_prompt_with_character(main_character, actions, settings, styles) for _ in range(5)]
selected_character = main_character

for i, prompt in enumerate(prompts_list):
    print(f"\n🎬 Prompt {i+1}: {prompt}")



# -------------------
# Video Generation Using HunyuanVideoPipeline
# -------------------
import torch
from diffusers import HunyuanVideoPipeline, HunyuanVideoTransformer3DModel, BitsAndBytesConfig
model_id = "hunyuanvideo-community/HunyuanVideo"

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
transformer = HunyuanVideoTransformer3DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)

# Initialize pipeline
pipe = HunyuanVideoPipeline.from_pretrained(
    model_id, 
    transformer=transformer, 
    torch_dtype=torch.float16
)
pipe.vae.enable_tiling()        # Memory optimization
pipe.enable_model_cpu_offload() # Offload to CPU if needed

all_frames = []

for i, prompt in enumerate(prompts_list):
    print(f"🎬 Generating segment {i+1}/{len(prompts_list)}: {prompt}")
    
    output = pipe(
        prompt=prompt,
        height=320,
        width=512,
        num_frames=100,          # ~4 sec at 15 fps
        num_inference_steps=30
    ).frames
    
    all_frames.append(output)

import numpy as np
np.save("debug_video_frames.npy", all_frames)
print("🧪 Raw frames saved to debug_video_frames.npy")

# -------------------
# Save Video to Disk (OpenCV)
# -------------------
# Flatten double-nested all_frames to a single list of PIL.Image.Imageimport cv2
import numpy as np
from PIL import Image
import cv2


video_path = VIDEO_FILE
fps = 15

frames = []
for segment in all_frames:
    for sequence in segment:
        for frame in sequence:
            if isinstance(frame, Image.Image):
                frames.append(frame)

if not frames:
    raise ValueError("No frames found to write into video. Please check generation pipeline.")


width, height = frames[0].size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

for frame in frames:
    np_frame = np.array(frame)
    bgr_frame = cv2.cvtColor(np_frame, cv2.COLOR_RGB2BGR)
    video_writer.write(bgr_frame)

video_writer.release()
print(f"✅ Video saved as {video_path}")





